<H2>Environment Setup</H2>

In [1]:
%pip install pandas
%pip install numpy
%pip install tensorflow
%pip install pyarrow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf

In [3]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [4]:
path_zip = tf.keras.utils.get_file(
    origin='https://zenodo.org/record/4964287/files/ECAD_DATA.zip',
    fname='DATA.zip',
    cache_subdir='ECAD',
    extract=True)

In [5]:
path_ecad = os.path.dirname(path_zip)

path_oslo = os.path.join(path_ecad, 'OSLO')
path_oslo

'C:\\Users\\ahmar\\.keras\\ECAD\\OSLO'

In [6]:
list_files_oslo = os.listdir(path_oslo)
list_files_oslo

['CC_STAID000193.txt',
 'DD_STAID000193.txt',
 'elements.txt',
 'FG_STAID000193.txt',
 'FX_STAID000193.txt',
 'HU_STAID000193.txt',
 'metadata.txt',
 'PP_STAID000193.txt',
 'QQ_STAID000193.txt',
 'RR_STAID000193.txt',
 'SD_STAID000193.txt',
 'sources.txt',
 'SS_STAID000193.txt',
 'stations.txt',
 'TG_STAID000193.txt',
 'TN_STAID000193.txt',
 'TX_STAID000193.txt']

<H2>Extract Transform Load</H2>

In [7]:
def read_data(file_name):
    """
    """
    header_line_number = None
    # open file
    with open(file_name, 'r') as file:
        # find header
        for line_number, line_text in enumerate(file):
            if 'SOUID' in line_text:
                header_line_number = line_number
    # read csv
    df = pd.DataFrame()
    if header_line_number:
        df = pd.read_csv(file_name, header=header_line_number, skipinitialspace=True, index_col='DATE', na_values='-9999', skip_blank_lines=False)
    return df

In [8]:
variables = {'CC' : 'CLOUD COVER',
             'DD' : 'WIND DIRECTION',
             'FG' : 'WIND SPEED',
             'FX' : 'WIND GUST',
             'HU' : 'HUMIDITY',
             'PP' : 'SEA LEVEL PRESSURE',
             'QQ' : 'GLOBAL RADIATION',
             'RR' : 'PRECIPITATION AMOUNT',
             'SS' : 'SUNSHINE',
             'TG' : 'MEAN TEMPERATURE',
             'TN' : 'MINIMUM TEMPERATURE',
             'TX' : 'MAXIMUM TEMPERATURE'}

In [9]:
data = None

for file_name in list_files_oslo:
    if file_name[0:2] in variables:
        df = read_data(os.path.join(path_oslo, file_name))
        df.drop('SOUID', inplace=True, axis=1)
        if data is None:
            data = df
        else:
            data = pd.merge(left=data, right=df, how='outer', on='DATE')

In [10]:
data.head().T

DATE,19370301,19370302,19370303,19370304,19370305
CC,NaN,NaN,NaN,NaN,NaN
Q_CC,9.0,9.0,9.0,9.0,9.0
DD,0.0,0.0,0.0,0.0,0.0
Q_DD,0.0,0.0,0.0,0.0,0.0
FG,NaN,NaN,NaN,NaN,NaN
Q_FG,9.0,9.0,9.0,9.0,9.0
FX,NaN,NaN,NaN,NaN,NaN
Q_FX,9.0,9.0,9.0,9.0,9.0
HU,NaN,NaN,NaN,NaN,NaN
Q_HU,9.0,9.0,9.0,9.0,9.0


In [11]:
data.shape

(44591, 24)

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44591 entries, 19370301 to 19301231
Data columns (total 24 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   CC      25658 non-null  float64
 1   Q_CC    30712 non-null  float64
 2   DD      30462 non-null  float64
 3   Q_DD    32963 non-null  float64
 4   FG      25658 non-null  float64
 5   Q_FG    32963 non-null  float64
 6   FX      14334 non-null  float64
 7   Q_FX    32963 non-null  float64
 8   HU      25658 non-null  float64
 9   Q_HU    30712 non-null  float64
 10  PP      23466 non-null  float64
 11  Q_PP    30712 non-null  float64
 12  QQ      24172 non-null  float64
 13  Q_QQ    25568 non-null  float64
 14  RR      44285 non-null  float64
 15  Q_RR    44316 non-null  float64
 16  SS      24062 non-null  float64
 17  Q_SS    30712 non-null  float64
 18  TG      30740 non-null  float64
 19  Q_TG    30740 non-null  float64
 20  TN      30771 non-null  float64
 21  Q_TN    30771 non-null  float6

In [14]:
data.index = pd.to_datetime(data.index.astype(str))

In [15]:
qi = [column for column in data.columns if column[0:2] == 'Q_']
qi

['Q_CC',
 'Q_DD',
 'Q_FG',
 'Q_FX',
 'Q_HU',
 'Q_PP',
 'Q_QQ',
 'Q_RR',
 'Q_SS',
 'Q_TG',
 'Q_TN',
 'Q_TX']

In [16]:
data[qi] =data[qi].astype('Int64')

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 44591 entries, 1937-03-01 to 1930-12-31
Data columns (total 24 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   CC      25658 non-null  float64
 1   Q_CC    30712 non-null  Int64  
 2   DD      30462 non-null  float64
 3   Q_DD    32963 non-null  Int64  
 4   FG      25658 non-null  float64
 5   Q_FG    32963 non-null  Int64  
 6   FX      14334 non-null  float64
 7   Q_FX    32963 non-null  Int64  
 8   HU      25658 non-null  float64
 9   Q_HU    30712 non-null  Int64  
 10  PP      23466 non-null  float64
 11  Q_PP    30712 non-null  Int64  
 12  QQ      24172 non-null  float64
 13  Q_QQ    25568 non-null  Int64  
 14  RR      44285 non-null  float64
 15  Q_RR    44316 non-null  Int64  
 16  SS      24062 non-null  float64
 17  Q_SS    30712 non-null  Int64  
 18  TG      30740 non-null  float64
 19  Q_TG    30740 non-null  Int64  
 20  TN      30771 non-null  float64
 21  Q_TN    30771 non-

In [18]:
data['FG'] = data['FG'] / 10
data['FX'] = data['FX'] / 10
data['PP'] = data['PP'] / 10
data['RR'] = data['RR'] / 10
data['SS'] = data['SS'] / 10 
data['TG'] = data['TG'] / 10
data['TN'] = data['TN'] / 10
data['TX'] = data['TX'] / 10

In [19]:
data.rename(columns={'CC' : 'CLOUD COVER (okta)',
                     'DD' : 'WIND DIRECTION (degrees)',
                     'FG' : 'WIND SPEED (m/s)',
                     'FX' : 'WIND GUST (m/s)',
                     'HU' : 'RELATIVE HUMIDITY',
                     'PP' : 'SEA LEVEL PRESSURE (hPa)',
                     'QQ' : 'GLOBAL RADIATION (W/m2)',
                     'RR' : 'PRECIPITATION AMOUNT (mm)',
                     'SS' : 'SUNSHINE (hours)',
                     'TG' : 'MEAN TEMPERATURE (C)',
                     'TN' : 'MINIMUM TEMPERATURE (C)',
                     'TX' : 'MAXIMUM TEMPERATURE (C)'},
            inplace=True)

In [20]:
data.sample(10).T

DATE,1991-12-13,1978-02-14,1987-11-20,1986-11-18,1973-02-07,1978-08-18,2013-10-04,1957-07-28,1997-09-28,1915-07-12
CLOUD COVER (okta),5.0,8.0,8.0,7.0,3.0,5.0,8.0,7.0,7.0,NaN
Q_CC,0,0,0,0,0,0,0,0,0,<NA>
WIND DIRECTION (degrees),350.0,20.0,40.0,200.0,350.0,230.0,179.0,270.0,210.0,NaN
Q_DD,0,0,0,0,0,0,0,0,0,<NA>
WIND SPEED (m/s),1.9,6.7,2.4,3.4,4.8,1.7,1.5,0.9,2.2,NaN
Q_FG,0,0,0,0,0,0,0,0,0,<NA>
WIND GUST (m/s),14.4,NaN,10.3,12.9,NaN,NaN,7.4,NaN,7.2,NaN
Q_FX,0,9,0,0,9,9,0,9,0,<NA>
RELATIVE HUMIDITY,74.0,71.0,80.0,86.0,49.0,76.0,84.0,89.0,89.0,NaN
Q_HU,0,0,0,0,0,0,0,0,0,<NA>


In [21]:
data.to_parquet('..\data\owf.Extract.01.parquet', index=True)